In [19]:
%pip install --upgrade steamship pydantic

  Using cached pydantic-1.10.2-cp38-cp38-macosx_11_0_arm64.whl (2.6 MB)
You should consider upgrading via the '/Users/edwardbenson/steamship-apps/chat-analytics-app/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [49]:
from steamship import Steamship
from datetime import datetime
from enum import Enum
from typing import Optional, List

from pydantic import BaseModel, Field, parse_obj_as

# chat-analytics Demo - Steamship Python Client

This notebook provides an example of using `chat-analytics` from Steamship's Python client.

To authenticate with Steamship, install the Steamship CLI with:

```bash
> npm install -g @steamship/cli
```

And then login with:

```bash
> ship login
```

Note that this notebook uses the following Python dependencies not included in `requirements.txt`:

* pandas

In [50]:
# This should match the value found in the `handle` field of `steamship.json`
PACKAGE_HANDLE="chat-analytics"

# The handle of the particular instance of the package we'll be working with.
INSTANCE_HANDLE = "my-instance-05"

In [51]:
# Create an instance of the package. Credentials will be loaded from ~/.steamship.json
instance = Steamship.use(PACKAGE_HANDLE, INSTANCE_HANDLE)

print(
    f"Created package instance {instance.handle} of {PACKAGE_HANDLE} in workspace {instance.client.config.space_handle or 'default'}")

Created package instance my-instance-05 of chat-analytics in workspace my-instance-05


In [52]:
# These model classes contain the data shape that the chat-analytics app expects as input/output
# They can be found in src/api_spec.py of the chat-analytics package on GitHub

class Intent(str, Enum):
    """Intent of a chat message."""

    SALUTATION = "Salutation"
    PRAISE = "Praise"
    COMPLAINT = "Complaint"
    QUESTION = "Question"
    REQUEST = "Request"
    EXPLANATION = "Explanation"


class Sentiment(str, Enum):
    """Sentiment of a chat message."""

    POSITIVE = "Positive"
    NEGATIVE = "Negative"
    NEUTRAL = "Neutral"

class Message(BaseModel):
    """Structured representation of a chat message."""

    message_id: str = Field(example="001")
    timestamp: datetime
    user_id: str = Field(example="u001")
    text: str = Field(example="Hello. This is a message.")
    sentiment: Optional[Sentiment]
    intent: Optional[Intent]
    root_message_id: Optional[str] = Field(example="001")

    def dict(self, format_dates: bool = False, format_enums: bool = False, **kwargs):
        """Transform object into a dictionary."""
        output = super().dict(**kwargs)
        for k, v in output.items():
            if format_dates and isinstance(v, datetime):
                output[k] = v.isoformat()
            if format_enums and isinstance(v, Enum):
                output[k] = v.value
        return output


In [53]:
# Here is an example chat, using the model classes above

MESSAGES = [
    Message(
        message_id="0",
        timestamp=datetime(2022, 6, 15, 16, 18, 33, 100),
        user_id="1",
        text="Hi Team!",
        intent=Intent.SALUTATION,
    ),
    Message(
        message_id="1",
        timestamp=datetime(2022, 6, 15, 16, 18, 33, 960),
        user_id="1",
        text="Thanks for getting back to us on the styling issue "
        "we had last week. Font colours are so important for productivity.",
        intent=Intent.PRAISE,
    ),
    Message(
        message_id="2",
        timestamp=datetime(2022, 6, 15, 16, 18, 34, 450),
        user_id="1",
        text="I noticed the Slack bot being very verbose lately",
    ),
    Message(
        message_id="3",
        timestamp=datetime(2022, 6, 15, 16, 18, 34, 990),
        user_id="1",
        text="Is there a way to decrease the verbosity level?",
    ),
    Message(
        message_id="4",
        timestamp=datetime(2022, 6, 15, 16, 18, 36, 211),
        user_id="1",
        text="I want ab.bot to ignore thank you messages and stop "
        "asking our customers to assign messages to threads.",
    ),
    Message(
        message_id="5",
        timestamp=datetime(2022, 6, 15, 16, 18, 42, 97),
        user_id="1",
        text="Our clients are not technical so working with threads is difficult.",
    ),
    Message(
        message_id="6",
        timestamp=datetime(2022, 6, 15, 16, 18, 44, 155),
        user_id="1",
        text="Thanks again, looking forward to your response!",
    ),
    Message(
        message_id="7",
        timestamp=datetime(2022, 6, 15, 16, 18, 44, 822),
        user_id="1",
        text="Oh, before I forget. Is there a settings to change the font size?",
    ),
]

In [54]:
# Now let's use the package.
# We'll call the "analyze" method, passing it the MESSAGES list formatted as JSON.
response = instance.post(
    "analyze",
    chat_stream=[message.dict(format_dates=True, format_enums=True) for message in MESSAGES],
)

output_stream = parse_obj_as(List[Message], response.data["chat_stream"])

ERROR:root:Client received error from server: [ERROR - POST /analyze] [ERROR - POST /tag] Unable to initialize plugin/app. | [Errno 2] No such file or directory: '/var/task/.steamship/secrets.toml'


SteamshipError: [ERROR - POST /analyze] [ERROR - POST /tag] Unable to initialize plugin/app. | [Errno 2] No such file or directory: '/var/task/.steamship/secrets.toml'

In [26]:
# The output chat stream has additional fields completed:
# - The intent